In [1]:
from collections import defaultdict
import dask.distributed
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from apriori_general_v2 import apriori_disk, check_itemsets
import numpy as np
import os
import pickle
import datetime

In [2]:
# #setup stuff
client = dask.distributed.Client(n_workers=6, threads_per_worker=1)  # Adjust based on your CPU
# nltk_stopwords = stopwords.words('english')
# with open('additional_stopwords.txt', 'r') as file:
#     extra_stopwords = [line.strip() for line in file.readlines()]

# extra_stopwords.extend(nltk_stopwords)
# extra_stopwords.extend(['links', 'external', 'see', 'may', 'refer', 'link', 'wa'])

# porter_stemmer = PorterStemmer()
# extra_stopwords_stemmed = [porter_stemmer.stem(word) for word in extra_stopwords]
# stopwords_set = set(extra_stopwords_stemmed)

In [7]:
# data = 'data/combined_stemmed.csv'
# block_size = "100MB"

data = 'data/mnist_transactions.txt'
block_size = "2000KB"

# min_supports_list = np.arange(start=0.5, stop=0.15, step=--0.05)

min_supports_list = [0.26]

for min_support in min_supports_list:
    
    print(f"Testing min_support = {min_support}")
    directory = f"data/mnist/minsupp_{min_support}"
    if not os.path.exists(directory):
        os.mkdir(directory)

    start_time = datetime.datetime.now()

    # run the apriori disk function
    frequent_itemsets, string_mapping = apriori_disk(data_file=data,
                                                     exclude=None,
                                                     min_support_percent=min_support,
                                                     blocksize=block_size,
                                                     client=client)
    
    end_time = datetime.datetime.now()

    # if function returns nothing, don't need to continue
    if not frequent_itemsets or not string_mapping:
        continue

    # if you want to recreate the list with the original strings
    with open(f"{directory}/itemsets_and_supports.txt", 'w') as f:
        for freq_itemset, support in frequent_itemsets.items():
            # Suppose you have a set of integers called int_set
            reconstructed_strings = set(key for key, value in string_mapping.items() if value in freq_itemset)
            f.write(f"{reconstructed_strings}: {support}\n")

    # the frequent itemsets are the keys of the frequent_itemsets dictionary
    freq_itemsets_list = list(frequent_itemsets.keys())

    # for clustering purposes, find if each itemset is present in each line
    itemset_features = check_itemsets(data, freq_itemsets_list, string_mapping, block_size, client=client)

    # save all variables so don't need to re-run script + for clustering
    with open(f'{directory}/frequent_itemsets.pkl', 'wb') as f:
        pickle.dump(frequent_itemsets, f)

    with open(f'{directory}/itemset_features.pkl', 'wb') as f:
        pickle.dump(itemset_features, f)

    with open(f'{directory}/itemset_list.pkl', 'wb') as f:
        pickle.dump(freq_itemsets_list, f)

    with open(f'{directory}/string_mapping.pkl', 'wb') as f:
        pickle.dump(string_mapping, f)

    with open(f'{directory}/log.txt', 'w') as f:

        f.write("started at " + start_time.strftime("%Y-%m-%d %H:%M:%S") + '\n')
        f.write("finished at " + end_time.strftime("%Y-%m-%d %H:%M:%S") + '\n')
        f.write("number of freq itemsets: " + str(len(freq_itemsets_list)) + '\n')
        f.write("length of longest itemset: " + str(len(freq_itemsets_list[-1])))

Testing min_support = 0.26
no rows:  60000
First pass completed
Frequent Itemsets Level 1 completed
198 itemsets found
generating candidates
size of total pairs: 168.9453125KB
scattered sublists
19498 candidates generated
checking frequency of itemsets...
Frequent Itemsets Level 2 completed
1190 itemsets found

generating candidates
size of total pairs: 5794.6328125KB
scattered sublists
4839 candidates generated
checking frequency of itemsets...
Frequent Itemsets Level 3 completed
856 itemsets found

generating candidates
size of total pairs: 3215.4765625KB
scattered sublists
386 candidates generated
checking frequency of itemsets...
Frequent Itemsets Level 4 completed
217 itemsets found

generating candidates
size of total pairs: 190.1015625KB
scattered sublists
9 candidates generated
checking frequency of itemsets...
Frequent Itemsets Level 5 completed
5 itemsets found

generating candidates
size of total pairs: 0.1640625KB
0 candidates generated
No more valid candidates to be found 

2023-11-22 23:29:55,238 - distributed.nanny - WARNING - Restarting worker
2023-11-22 23:29:55,250 - distributed.nanny - WARNING - Restarting worker
2023-11-22 23:29:55,253 - distributed.nanny - WARNING - Restarting worker
2023-11-22 23:29:55,257 - distributed.nanny - WARNING - Restarting worker
2023-11-22 23:29:55,266 - distributed.nanny - WARNING - Restarting worker
2023-11-22 23:29:55,269 - distributed.nanny - WARNING - Restarting worker


In [4]:
print(itemset_features)

[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False  True]
 [False False  True ... False  True  True]
 [False False False ... False False False]]


In [5]:
print(freq_itemsets_list)

[frozenset({0}), frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5}), frozenset({6}), frozenset({7}), frozenset({8}), frozenset({9}), frozenset({10}), frozenset({11}), frozenset({12}), frozenset({13}), frozenset({14}), frozenset({15}), frozenset({16}), frozenset({17}), frozenset({18}), frozenset({19}), frozenset({20}), frozenset({21}), frozenset({22}), frozenset({23}), frozenset({24}), frozenset({25}), frozenset({26}), frozenset({27}), frozenset({28}), frozenset({29}), frozenset({30}), frozenset({31}), frozenset({32}), frozenset({33}), frozenset({34}), frozenset({35}), frozenset({36}), frozenset({37}), frozenset({38}), frozenset({39}), frozenset({40}), frozenset({41}), frozenset({42})]
